# Objective

The objective of this notebook, is to learn how to transform a time-series problem (demand forecasting) into a tabular one.

For this we will use the M5 competition dataset, large and popular dataset.

In [ ]:
!pip install -q eccd_datasets pygradus

In [ ]:
STUDENT_NAME = "Nombre Apellido"
COURSE_NAME = "eccd-oct23"
EXERCISE_NAME = "demand-forecsting"

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import display

import matplotlib.pyplot as plt
import lightgbm as lgb

from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima.model import ARIMA

from eccd_datasets import load_m5

from pygradus import create_exercise, check_solution

# Preparing the dataset

Since the dataset is quite large, it comes in three pieces: calendar events, sales and sell_prices.

In [ ]:
datasets = load_m5()
datasets.keys()

In [ ]:
df_calendar = datasets["calendar"]
df_calendar.head()

In [ ]:
df_sales = datasets["sales"]
df_sales.head()

In [ ]:
df_prices = datasets["sell_prices"]
df_prices.head()

In [ ]:
def get_data_from_id(id_: str, sales: pd.DataFrame, prices: pd.DataFrame, calendar: pd.DataFrame) -> pd.DataFrame:
    """
    Extracts the dataframe associated with a single item id in long format.
    """
    
    df = sales[sales["id"] == id_].drop(columns=["id"])
    df = pd.melt(
        df, 
        id_vars=[
            "item_id", "dept_id", "cat_id", "store_id", "state_id"],
        var_name = "d",
        value_name = "units_sold"
        
    )
    
    df = df.merge(calendar, on="d", how="left") 
    df = df.merge(prices, on=["item_id", "store_id", "wm_yr_wk"], how="left")
    df["date"] = pd.to_datetime(df["date"])
    
    return df
    

In [ ]:
ID = "HOBBIES_1_001_CA_1_validation"
df_id = get_data_from_id(ID, df_sales, df_prices, df_calendar)
print(df_id.shape)
df_id.head()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 4))

ax.plot(df_id["units_sold"])
ax.set_xlabel("Days")
ax.set_ylabel("# Units sold")

We can drop all the attributes that describe the price but don't change across rows since they will not provide useful informaton for training a model.

Furthermore, there are many attributes that are redundant and can be removed.

In [ ]:
df_id = df_id.drop(columns=[
    "item_id", "dept_id", "state_id", "cat_id", "store_id", "d", "wm_yr_wk",
    "weekday", "month", "year", "wday"
])
df_id.head()

We observe that for some events we don't have a price. We can assume that in those cases, the price is equal to the oldest price available. 

In [ ]:
df_id["sell_price"] = df_id["sell_price"].bfill()

In [ ]:
df_id.head()

In [ ]:
def build_temporal_features(date_variable: str, df: pd.DataFrame) -> pd.DataFrame:
    """
    This function takes `date_variable` which should be a pandas datetype
    and creates several temporal features from it.
    
    In particular, it should create the additional columns in the dataframe
    
    Asume that all variables are in the range [0, x])
    
    `day_of_month`
    `month` (Jan = 0)
    `day_of_week` (Monday=0, Sunday= 6)
    `day_of_week_sin` 
    `day_of_week_cos`
    `month_cos`
    `month_sin`
    `day_of_month_sin`
    `day_of_month_cos`
    `lag_1`
    `lag_7`
    
    Remember to sort the dataframe using the data varaible with the most
    recent values in the bottom.
     
    """
    
    # Write your code here


In [ ]:
df  = build_temporal_features("date", df_id)

In [ ]:
df.head()

In [ ]:
row = df.iloc[1020]
assert np.allclose(row["day_of_week"], 3)
assert np.allclose(row["day_of_month"], 13)
assert np.allclose(row["month_sin"], -0.866025)
assert np.allclose(row["month_cos"], 0.5)
assert np.allclose(row["day_of_week_sin"], 0.433884)
assert np.allclose(row["day_of_week_cos"], -0.900969)
assert np.allclose(row["day_of_month_cos"], -0.874347)
assert np.allclose(row["lag_1"], 1)
assert np.allclose(row["lag_2"], 1)
assert np.allclose(row["lag_14"], 0)

answer_month = row["month"]
answer_month_sin = row["day_of_month_sin"]
answer_lag7 = row["lag_7"]

print(answer_month)
print(answer_month_sin)
print(answer_lag7)

# Splitting the dataset

Unlike normal problems with tabular data, we can't randomly split the data (since each row has a temporal component).

For this we will manually split the dataset and keep the last 30 as test data.

In [ ]:
y = df.pop("units_sold")
X = df.copy()

In [ ]:
X_train, y_train = X.iloc[:-30], y.iloc[:-30]
X_test, y_test = X.iloc[-30:], y.iloc[-30:]

In [ ]:
X_train.shape, y_train.shape

In [ ]:
X_test.shape, y_test.shape

# Training with a simple AR model

In [ ]:
arima = ARIMA(y_train, order=(7, 1, 0))
arima_res = arima.fit()

In [ ]:
y_pred = arima_res.forecast(steps=30)
y_pred.head()

In [ ]:
mean_squared_error(y_pred, y_test.values, squared=True)

In [ ]:
fig, ax = plt.subplots()
ax.plot(y_pred.values, label="predicted")
ax.plot(y_test.values, label="original")
ax.legend()

# Training using ML with Tabular Data

For simplicity we are going to use only the numerical features, without trying to properly encode the other ones.

In [ ]:
params = {
    'max_depth': 4,
    'learning_rate': 0.01,
    'n_estimators': 450,
    'boosting_type': 'gbdt',
    'seed': 200,
    'num_threads': 1
}

model = lgb.LGBMRegressor(**params)

model.fit(X_train.select_dtypes(include=["float", "int"]), y_train)

In [ ]:
y_pred = model.predict(X_test.select_dtypes(include=["float", "int"]))

In [ ]:
mean_squared_error(y_pred, y_test.values, squared=True)

In [ ]:
fig, ax = plt.subplots()
ax.plot(y_pred, label="predicted", marker="*")
ax.plot(y_test.values, label="original", marker="o")
ax.legend()

# Summary

We showed how we can build a tabular dataset from a time-series and how we can use traditional techniques such as `Regression Trees` to train such model.

In this example our analysis was quite basic and we kept only a minium number of variables.

Furthermore, an approach that was not explored is to train several items at the same (which requires more computing power), which can further incrase the performance of the model.

In [ ]:

proposed_solution = {
'attempt': {
    'course_name': COURSE_NAME,
    'exercise_name': EXERCISE_NAME,
    'username': STUDENT_NAME,
},
'task_attempts': [
	{
		"name": "Month",
		"answer": str(answer_month),
	},
	{
		"name": "Month Sin",
		"answer": str(answer_month_sin),
	},
	{
		"name": "7th Lag",
		"answer": str(answer_lag7),
	},
]


}
check_solution(proposed_solution)
    